In [ ]:
using CSV
using DataFrames
using StringEncodings
using Plots
using LinearAlgebra
using Statistics
using StatsBase
using Printf
using ForwardDiff
using Random
using NLopt
using StatsPlots
using CategoricalArrays
# using BenchmarkTools

In [ ]:
for file in readdir("functions/entry_exit/")
    include("functions/entry_exit/" * file)
end

In [ ]:
data_raw = data = CSV.File(
    open(read, "data/entry_exit_application/data_hospital_chapter6.csv", enc"UTF-8"),
    missingstring = ["NA", ""],
    ) |> DataFrame
first(data, 5)

In [ ]:
data_cleaned = data_raw[:, :];
replace!(data_cleaned.Management, missing => "");
data_cleaned[!, :DaigakuDum] = in(["公立大学法人", "国（国立大学法人）", "私立学校法人"]).(data_cleaned.Management);
data_cleaned[!, :ZeroBedDum] = (data_cleaned[:, :NumBeds] .== 0);

data_cleaned[!, :NumBeds] = data_cleaned[:, :NumBeds] ./ 100.0;
data_cleaned[!, :LogNumBeds] = log.(data_cleaned[:, :NumBeds] .+ 0.01);
data_cleaned[!, :Population] = data_cleaned[:, :Population] ./ 1e+6;
data_cleaned[!, :Menseki] = data_cleaned[:, :Menseki] ./ 100.0;
data_cleaned[!, :TaxableIncome] = data_cleaned[:, :TaxableIncome] ./ 1000.0;
data_cleaned[!, :LogPop] = log.(data_cleaned[:, :Population]);
data_cleaned[!, :LogIncome] = log.(data_cleaned[:, :TaxableIncome]);

## 4


In [ ]:
listVars = [
    "Kyukyu", "Kinou", "Sien", "Hyoka", "DepNeurology",
    "DepNeurosurgery", "NumBeds", "ZeroBedDum", "DaigakuDum"
]
table_mean = combine(
    groupby(data_cleaned, :MRIOwnDum),
    [Symbol(variable) for variable in listVars] .=> 
    x -> mean(skipmissing(x))
);
table_sd = combine(
    groupby(data_cleaned, :MRIOwnDum),
    [Symbol(variable) for variable in listVars] .=> 
    x -> std(skipmissing(x))
);

DataFrame(
    variable = listVars,
    meanOwnMRI = Matrix(table_mean)[2, 2:end],
    sdOwnMRI = Matrix(table_sd)[2, 2:end],
    meanNotOwnMRI_ = Matrix(table_mean)[1, 2:end],
    sdNotOwnMRI = Matrix(table_sd)[1, 2:end]
)

In [ ]:
vcat(
    calculateShareOwnMRI(data_cleaned),
    calculateShareOwnMRI(dropmissing(data_cleaned, :Kyukyu)[dropmissing(data_cleaned, :Kyukyu).Kyukyu .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :Sien)[dropmissing(data_cleaned, :Sien).Sien .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :Hyoka)[dropmissing(data_cleaned, :Hyoka).Hyoka .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :DepNeurology)[dropmissing(data_cleaned, :DepNeurology).DepNeurology .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :DepNeurosurgery)[dropmissing(data_cleaned, :DepNeurosurgery).DepNeurosurgery .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :LogNumBeds)[dropmissing(data_cleaned, :LogNumBeds).LogNumBeds .>= log(1.2), :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :DaigakuDum)[dropmissing(data_cleaned, :DaigakuDum).DaigakuDum .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_cleaned, :ZeroBedDum)[dropmissing(data_cleaned, :ZeroBedDum).ZeroBedDum .== 1, :]),
)

## 5


In [ ]:
data_processed_pre = dropmissing(data_cleaned[:, [
            :CityCode, :Kyukyu, :Kinou, :Sien, :Hyoka,
            :DepNeurology, :DepNeurosurgery, :LogNumBeds,
            :ZeroBedDum, :DaigakuDum,
            :Menseki, :LogPop, :LogIncome, :MRIOwnDum
            ]]);
first(data_processed_pre, 5)

In [ ]:
function processDataForBerryEst(df)
    
    data_processed = df[:, :];
    data_processed[!, :TieEntryOrder] = rand(nrow(df));
    
    transform!(
        groupby(data_processed, :CityCode),
        nrow => :numPotenHos,
        :MRIOwnDum => sum => :numEntryObs
    );
    sort!(data_processed, [:CityCode, :LogNumBeds, :TieEntryOrder], rev = [false, true, true]);
    transform!(groupby(data_processed, :CityCode), :numPotenHos => (x -> 1:length(x)) => :EntryOrderId);
    
    return data_processed
        
end

In [ ]:
Random.seed!(123)
data_processed = processDataForBerryEst(data_processed_pre);
first(data_processed, 5)

In [ ]:
NumPotenHos_max = 4;
data_processed = data_processed[data_processed.EntryOrderId .<= NumPotenHos_max, :];
transform!(
    groupby(data_processed, :CityCode),
    :MRIOwnDum => sum => :numEntryObs,
    nrow => :numPotenHos
);
data_processed[!, :Const] .= 1;
vcat(
    calculateShareOwnMRI(data_processed),
    calculateShareOwnMRI(dropmissing(data_processed, :Kyukyu)[dropmissing(data_processed, :Kyukyu).Kyukyu .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :Sien)[dropmissing(data_processed, :Sien).Sien .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :Hyoka)[dropmissing(data_processed, :Hyoka).Hyoka .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :DepNeurology)[dropmissing(data_processed, :DepNeurology).DepNeurology .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :DepNeurosurgery)[dropmissing(data_processed, :DepNeurosurgery).DepNeurosurgery .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :LogNumBeds)[dropmissing(data_processed, :LogNumBeds).LogNumBeds .>= log(1.2), :]),
    calculateShareOwnMRI(dropmissing(data_processed, :DaigakuDum)[dropmissing(data_processed, :DaigakuDum).DaigakuDum .== 1, :]),
    calculateShareOwnMRI(dropmissing(data_processed, :ZeroBedDum)[dropmissing(data_processed, :ZeroBedDum).ZeroBedDum .== 1, :]),
)

In [ ]:
numSim = 100;

uniqueCityCode = unique(data_processed.CityCode);
numCity = length(uniqueCityCode);
numHos = nrow(data_processed);
numEntryObs = combine(
    groupby(data_processed, :CityCode),
    :MRIOwnDum => sum => :numEntryObs
).numEntryObs;
numEntryObs = combine(
    groupby(data_processed, :CityCode),
    :MRIOwnDum => sum => :numEntryObs
).numEntryObs;

cityIndex = data_processed.CityCode .== uniqueCityCode';

In [ ]:
u_m0 = cityIndex * randn(numCity, numSim);
u_mIm = randn(numHos, numSim);

In [ ]:
param = param_init

alpha = param[1:8];
beta = param[9:12];
delta = param[13];
rho = param[14];

variableProfit = Matrix(data_processed[:, [:Const, :Menseki, :LogPop, :LogIncome]]) * beta .+ rho .* u_m0;
fixedCost = Matrix(data_processed[:, [
    :Kyukyu, :Sien, :Hyoka, :DepNeurology, :DepNeurosurgery, :LogNumBeds, :ZeroBedDum, :DaigakuDum
    ]]) * alpha .- (sqrt(1.0 - rho^2) .* u_mIm);
profitExcludeCompetition = variableProfit - fixedCost;

data_processed[data_processed.CityCode .== 1101, :]

entry_decision_df = hcat(
    data_processed[:, [:CityCode]], 
    DataFrame((profitExcludeCompetition .- delta .* log(3) .>= 0), :auto)
    );

transform(
    groupby(entry_decision_df, :CityCode), 
    Not(:CityCode) .=> sum
    )[:, r"_sum"] |> Matrix

cityIndex[:, 1]

city = 1

@time max.([
    (sum(
            view(profitExcludeCompetition, cityIndex[:, city], :) .- delta .* log(i) .>= 0, dims = 1
        ) .>= i) .* i
        for i in 1:4
]...)

@time reduce(
    (x, y) -> max.(x, y),
    [
        (sum(
            view(profitExcludeCompetition, cityIndex[:, city], :) .- delta .* log(i) .>= 0, dims = 1
            ) .>= i) .* i
        for i in 1:4
    ]
)

function each_entry_func(i)
    entry_decision_df = hcat(
        data_processed[:, [:CityCode]], 
        DataFrame((profitExcludeCompetition .- delta .* log(i) .>= 0), :auto)
        );

    return (Matrix(
        combine(
            groupby(entry_decision_df, :CityCode), 
            Not(:CityCode) .=> sum
            )[:, Not(:CityCode)]
            ) .>= i) .* i
end

@time each_entry_mat = max.(
    each_entry_func(1),
    each_entry_func(2),
    each_entry_func(3),
    each_entry_func(4)
);



aa = cityIndex[:, city]
profitExcludeCompetition[aa, :]

tt = profitExcludeCompetition[aa, k] .- delta .* log.(each_entry_mat[city, k] + 1) .>= 0

tt


k = 1


In [ ]:
param_init = [
    -0.612340533,   -5.525423772,   -0.505275676,   
    -0.32531026,    -1.04162392,    -0.991878025,   
    -3.87040966,    -1.272714254,   2.684741676,    
    0.040555764,    0.426448612,    -1.399627382,   
    0.990975782,    0.958075433
];

function calculateBerryObjective(
        param,
        df::DataFrame,
        u_m0::Matrix{Float64},
        u_mIm::Matrix{Float64},
        numEntryObs::Vector{Int}
    )
    
    alpha = param[1:8];
    beta = param[9:12];
    delta = param[13];
    rho = param[14];
    
    variableProfit = Matrix(df[:, [:Const, :Menseki, :LogPop, :LogIncome]]) * beta .+ rho .* u_m0;
    fixedCost = Matrix(df[:, [
        :Kyukyu, :Sien, :Hyoka, :DepNeurology, :DepNeurosurgery, :LogNumBeds, :ZeroBedDum, :DaigakuDum
        ]]) * alpha .- (sqrt(1.0 - rho^2) .* u_mIm);
    profitExcludeCompetition = variableProfit - fixedCost;

    function each_entry_func(i)
        entry_decision_df = hcat(
            df[:, [:CityCode]], 
            DataFrame((profitExcludeCompetition .- delta .* log(i) .>= 0), :auto)
            );
        return (Matrix(
            combine(
                groupby(entry_decision_df, :CityCode), 
                Not(:CityCode) .=> sum
                )[:, Not(:CityCode)]
                ) .>= i) .* i
    end
    
    each_entry_mat = max.(
        each_entry_func(1),
        each_entry_func(2),
        each_entry_func(3),
        each_entry_func(4)
    );
    n_exp = mean(each_entry_mat, dims = 2);
    
    diff = mean((numEntryObs .- n_exp).^2);

    return diff
    
end

In [ ]:
@time calculateBerryObjective(param_init, data_processed, u_m0, u_mIm, numEntryObs);

In [ ]:
#| echo: false
#| eval: false

# obj_for_Optim = TwiceDifferentiable(
#     x -> calculateBerryObjective(x, data_processed, u_m0, u_mIm, numEntryObs),
#     param_init;
#     autodiff = :forward
# );
# @time optim_res = Optim.optimize(
#     obj_for_Optim,
#     # x -> calculateBerryObjective(x, data_processed, u_m0, u_mIm, numEntryObs),
#     [repeat([-Inf], 13); -1.0],
#     [repeat([Inf], 13); 1.0],
#     param_init,
#     # NelderMead(),
#     Optim.Options(show_trace = false)
# );

# [param_init optim_res.minimizer]

In [ ]:
# @benchmark calculateBerryObjective(param_init, data_processed, u_m0, u_mIm, numEntryObs)
function obj_for_Optim(x::Vector, grad::Vector)
    if length(grad) != 0
        ForwardDiff.gradient!(grad, x -> calculateBerryObjective(x, data_processed, u_m0, u_mIm, numEntryObs), x)
    end
    return calculateBerryObjective(x, data_processed, u_m0, u_mIm, numEntryObs)
end

param_init = [
    -0.612340533,   -5.525423772,   -0.505275676,   
    -0.32531026,    -1.04162392,    -0.991878025,   
    -3.87040966,    -1.272714254,   2.684741676,    
    0.040555764,    0.426448612,    -1.399627382,   
    0.990975782,    0.958075433
];

opt = NLopt.Opt(:LN_NELDERMEAD, length(param_init))
opt.lower_bounds = [repeat([-Inf], 13); -1.0]
opt.upper_bounds = [repeat([Inf], 13); 1.0]

opt.min_objective = obj_for_Optim;
@time (minf, minx, ret) = NLopt.optimize(opt, param_init)

[param_init minx]

In [ ]:
numBootstrap = 100;
bootEstMat = zeros(length(minx), numBootstrap)

@time for bootIndex in 1:numBootstrap

    bootCitySample = sample(uniqueCityCode, numCity, replace = true);
    df_boot = reduce(
        vcat,
        [
            transform(
                data_processed[data_processed.CityCode .== bootCitySample[city] , :],
                :CityCode => (x -> city) => :CityCode
            )
            for city in 1:numCity
        ]
    );

    numHos_boot = nrow(df_boot);

    uniqueCityCode_boot = unique(df_boot.CityCode);
    cityIndex_boot = df_boot.CityCode .== uniqueCityCode';

    u_m0_boot = cityIndex_boot * randn(numCity, numSim);
    u_mIm_boot = randn(numHos_boot, numSim);

    numEntryObs_boot = combine(
        groupby(df_boot, :CityCode),
        :MRIOwnDum => sum => :numEntryObs
    ).numEntryObs;

    function obj_for_Optim(x::Vector, grad::Vector)
        if length(grad) != 0
            ForwardDiff.gradient!(grad, x -> calculateBerryObjective(x, df_boot, u_m0_boot, u_mIm_boot, numEntryObs_boot), x)
        end
        return calculateBerryObjective(x, df_boot, u_m0_boot, u_mIm_boot, numEntryObs_boot)
    end

    opt_boot = NLopt.Opt(:LN_NELDERMEAD, length(param_init))
    opt_boot.lower_bounds = [repeat([-Inf], 13); -1.0]
    opt_boot.upper_bounds = [repeat([Inf], 13); 1.0]

    opt_boot.min_objective = obj_for_Optim;
    (minf_boot, minx_boot, ret_boot) = NLopt.optimize(opt_boot, param_init)
    bootEstMat[:, bootIndex] .= minx_boot;

end


In [ ]:
[minx std(bootEstMat, dims = 2)];

## 6.2


In [ ]:
function simulateEntry(
    df, 
    delta, 
    profitExcludeCompetition
    )

    function each_entry_func(i)
        entry_decision_df = hcat(
            df[:, [:CityCode]], 
            DataFrame((profitExcludeCompetition .- delta .* log(i) .>= 0), :auto)
            );
        return (Matrix(
            combine(
                groupby(entry_decision_df, :CityCode), 
                Not(:CityCode) .=> sum
                )[:, Not(:CityCode)]
                ) .>= i) .* i
    end
    
    each_entry_mat = max.(
        each_entry_func(1),
        each_entry_func(2),
        each_entry_func(3),
        each_entry_func(4)
    );

    return vec(mean(each_entry_mat, dims = 2))

end

berry_est = minx;
alpha_est = berry_est[1:8];
beta_est = berry_est[9:12];
delta_est = berry_est[13];
rho_est = berry_est[14];

variableProfit = (
    Matrix(data_processed[:, [:Const, :Menseki, :LogPop, :LogIncome]]) * beta_est .+ 
    rho_est .* u_m0
);
fixedCost = (
    Matrix(data_processed[:, [
        :Kyukyu, :Sien, :Hyoka, :DepNeurology, 
        :DepNeurosurgery, :LogNumBeds, :ZeroBedDum, :DaigakuDum
        ]]) * alpha_est .- (sqrt.(1.0 - rho_est^2) .* u_mIm)
)

profitExcludeCompetition = variableProfit - fixedCost;

entryProb = simulateEntry(
    data_processed,
    delta_est,
    profitExcludeCompetition
);

In [ ]:
entryPred = reduce(vcat, entryProb) .> 0.5;

data_predicted_agg = DataFrame(
    Actual = numEntryObs,
    Predict = entryPred,
);

data_predicted = copy(data_processed);
data_predicted[!, :entryProb] = entryProb;
data_predicted[!, :entryPred] = entryPred;
data_predicted_agg = combine(
    groupby(data_predicted, :CityCode),
    :MRIOwnDum => sum => :Actual,
    :entryPred => sum => :Predict
);

data_predicted_agg = stack(data_predicted_agg, [:Actual, :Predict]);
data_predicted_sum = combine(groupby(data_predicted_agg, [:variable, :value]), nrow);
sort!(data_predicted_sum, [:variable, :value]);
groupedbar(
    data_predicted_sum.nrow, 
    group = data_predicted_sum.variable,
    xlabel = "value", 
    ylabel = "count",
    bar_width = 0.67,
    xticks = (1:5, 0:4),
    lw = 0
)

In [ ]:
#| echo: false
#| eval: false
 
# var_list = [:Kyukyu, :Sien, :Hyoka, :DepNeurology, :DepNeurosurgery, :DaigakuDum, :ZeroBedDum]
# actual_pred_MRI = map(x -> combine(data_predicted[data_predicted[:, x] .== 1, [:MRIOwnDum, :entryPred]], All() .=> sum), var_list);
# actual_pred_MRI = vcat(
#     combine(data_predicted[:, [:MRIOwnDum, :entryPred]], All() .=> sum),
#     reduce(vcat, actual_pred_MRI),
#     combine(data_predicted[data_predicted[:, :LogNumBeds] .>= log(1.2), [:MRIOwnDum, :entryPred]], All() .=> sum)
# )
# groupedbar(
#     Matrix(actual_pred_MRI), 
#     bar_position = :dodge, 
#     bar_width = 0.7,
#     xticks = (1:9, 1:9),

# )
# var_list = [:DepNeurology, :DepNeurosurgery, :ZeroBedDum, :DaigakuDum]
# DataFrame(
#     category = ["Total", "DepNeurology", "DepNeurosurgery", "ZeroBedDum", "DaigakuDum"],
#     mse = [
#         msd(data_predicted[:, :MRIOwnDum] * 1.0, data_predicted[:, :entryProb]);
#         map(i -> msd(data_predicted[data_predicted[:, i] .== 1, :MRIOwnDum] * 1.0, data_predicted[data_predicted[:, i] .== 1, :entryProb]), var_list)
#             ]
# )

## 7


In [ ]:
data_cf = copy(data_processed);
sort!(data_cf, [:CityCode, :DepNeurology, :DepNeurosurgery, :LogNumBeds, :TieEntryOrder], rev = [false, true, true, true, true]);
transform!(groupby(data_cf, :CityCode), :numPotenHos => (x -> 1:length(x)) => :EntryOrderId);
var_profit_cf = Matrix(data_cf[:, [:Const, :Menseki, :LogPop, :LogIncome]]) * beta_est .+ 
    rho_est .* u_m0;
fixedCost_cf = Matrix(data_cf[:, [:Kyukyu, :Sien, :Hyoka, :DepNeurology, :DepNeurosurgery, :LogNumBeds, :ZeroBedDum, :DaigakuDum]]) * alpha_est .-
    (sqrt.(1.0 - rho_est^2) .* u_mIm);

profitExcludeCompetition_cf = var_profit_cf - fixedCost_cf;
profitExcludeCompetition_cf[(data_cf.DepNeurology .== 1) .| (data_cf.DepNeurosurgery .== 1), :] .= 1e+5;

In [ ]:
EntryProb_cf = map(
    i -> simulateEntry(data_cf[data_cf.CityCode .== i, :], delta_est, profitExcludeCompetition_cf[data_cf.CityCode .== i, :]), 
    uniqueCityCode
);

In [ ]:
EntryPred_cf = reduce(vcat, EntryProb_cf) .> 0.5;
data_predicted_cf = copy(data_processed);
data_predicted_cf[!, :entryProb] = reduce(vcat, EntryProb_cf);
data_predicted_cf[!, :entryPred] = EntryPred_cf;
data_predicted_cf_agg = combine(
    groupby(data_predicted_cf, :CityCode),
    :MRIOwnDum => sum => :Actual,
    :entryPred => sum => :CounterFactual
);

data_predicted_cf_agg = stack(data_predicted_cf_agg, [:Actual, :CounterFactual]);
data_predicted_cf_sum = combine(groupby(
    vcat(data_predicted_agg, data_predicted_cf_agg[data_predicted_cf_agg.variable .== "CounterFactual", :]),
    [:variable, :value]
    ), nrow);

transform!(
    data_predicted_cf_sum, 
    :variable => 
    (x -> categorical(x; ordered = true, levels = ["Actual", "Predict", "CounterFactual"])) => 
    :variable
    )
sort!(data_predicted_cf_sum, [:variable, :value]);
groupedbar(
    data_predicted_cf_sum.nrow, 
    group = data_predicted_cf_sum.variable,
    xlabel = "value", 
    ylabel = "count",
    bar_width = 0.67,
    xticks = (1:5, 0:4),
    lw = 0
)

In [ ]:
vcat(
    combine(groupby(
        data_predicted_agg, :variable
        ), :value => sum),
    combine(groupby(
        data_predicted_cf_agg, :variable
        ), :value => sum) |> 
        filter(:variable => (x -> x .== "CounterFactual"))
)

In [ ]:
data_predicted_agg_wo_neuro = combine(
    groupby(
        filter([:DepNeurology, :DepNeurosurgery] => ((d1, d2) -> (d1 .== 1) .& (d2 .== 1)), data_predicted),
        :CityCode
        ),
    :MRIOwnDum => sum => :Actual,
    :entryPred => sum => :Predict
);

data_predicted_agg_wo_neuro = stack(data_predicted_agg_wo_neuro, [:Actual, :Predict]);
data_predicted_sum_wo_neuro = combine(groupby(data_predicted_agg_wo_neuro, [:variable, :value]), nrow);

data_predicted_cf_agg_wo_neuro = combine(
    groupby(
        # data_predicted_cf, 
        filter([:DepNeurology, :DepNeurosurgery] => ((d1, d2) -> (d1 .== 1) .& (d2 .== 1)), data_predicted_cf),
        :CityCode
        ),
    :MRIOwnDum => sum => :Actual,
    :entryPred => sum => :CounterFactual
);

data_predicted_cf_agg_wo_neuro = stack(data_predicted_cf_agg_wo_neuro, [:Actual, :CounterFactual]);

data_predicted_cf_sum_wo_neuro = combine(groupby(
    vcat(
        data_predicted_agg_wo_neuro, 
        data_predicted_cf_agg_wo_neuro[data_predicted_cf_agg_wo_neuro.variable .== "CounterFactual", :]
        ),
    [:variable, :value]
    ), nrow)

transform!(
    data_predicted_cf_sum_wo_neuro, 
    :variable => 
    (x -> categorical(x; ordered = true, levels = ["Actual", "Predict", "CounterFactual"])) => 
    :variable
    )
sort!(data_predicted_cf_sum_wo_neuro, [:variable, :value]);
groupedbar(
    data_predicted_cf_sum_wo_neuro.nrow, 
    group = data_predicted_cf_sum_wo_neuro.variable,
    xlabel = "value", 
    ylabel = "count",
    bar_width = 0.67,
    xticks = (1:5, 0:4),
    lw = 0
)

In [ ]:
vcat(
    combine(groupby(
        data_predicted_agg_wo_neuro, :variable
        ), :value => sum),
    combine(groupby(
        data_predicted_cf_agg_wo_neuro, :variable
        ), :value => sum) |> 
        filter(:variable => (x -> x .== "CounterFactual"))
)

In [ ]:
#| echo: false
#| eval: false

# EntryPred_cf = reduce(vcat, EntryProb_cf) .> 0.5;
# data_cf[!, :entryProb] = reduce(vcat, EntryProb_cf);
# data_cf[!, :entryPred] = EntryPred_cf;
# var_list = [:Kyukyu, :Sien, :Hyoka, :DepNeurology, :DepNeurosurgery, :DaigakuDum, :ZeroBedDum]
# actual_pred_MRI_cf = map(x -> combine(data_cf[data_cf[:, x] .== 1, [:MRIOwnDum, :entryPred]], All() .=> sum), var_list);
# actual_pred_MRI_cf = vcat(
#     combine(data_cf[:, [:MRIOwnDum, :entryPred]], All() .=> sum),
#     reduce(vcat, actual_pred_MRI_cf),
#     combine(data_cf[data_cf[:, :LogNumBeds] .>= log(1.2), [:MRIOwnDum, :entryPred]], All() .=> sum),
#     combine(data_cf[(data_cf[:, :DepNeurology] .== 0) .& (data_cf[:, :DepNeurosurgery] .== 0), [:MRIOwnDum, :entryPred]], All() .=> sum)
# )

# [actual_pred_MRI.EntryPred_sum actual_pred_MRI_cf.EntryPred_sum]